# Logistic Regression

In this notebook, we will learn how to apply Logistic regression for predicting the cooling load requirements (Y2) of buildings as a function of building parameters (Xs).

The attached dataset is taken from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency).

To run this code, you will need the following python packages:
* numpy
* pandas
* openpyxl
* scikit-learn

In [1]:
import numpy as np
import pandas as pd

In [ ]:
# !pip install openpyxl

In [3]:
# First, we load the dataset using pandas
df = pd.read_excel("Energy_Efficiency.xlsx", engine = 'openpyxl')
# Remove any unnamed columns (might occur due to difference in pandas readers)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# Remove any row with NaNs
df = df.dropna(how='all')
# Drop Y1 (as we only consider Y2 for classification)
df = df.drop('Y1', axis=1)

In [4]:
# next, we will split the dataframe into a training and testing splits with a 70% / 30% ratio
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.3, random_state=42) # Random is fixed for reproducability

In [5]:
df_train

,X1,X2,X3,X4,X5,X6,X7,X8,Y2
334,0.62,808.5,367.5,220.50,3.5,4,0.25,1,15.77
139,0.64,784.0,343.0,220.50,3.5,5,0.10,2,19.30
485,0.90,563.5,318.5,122.50,7.0,3,0.25,5,32.00
547,0.79,637.0,343.0,147.00,7.0,5,0.40,1,46.94
18,0.79,637.0,343.0,147.00,7.0,4,0.00,0,30.93
...,...,...,...,...,...,...,...,...,...
71,0.76,661.5,416.5,122.50,7.0,5,0.10,1,33.67
106,0.86,588.0,294.0,147.00,7.0,4,0.10,2,27.36
270,0.71,710.5,269.5,220.50,3.5,4,0.10,5,14.26
435,0.98,514.5,294.0,110.25,7.0,5,0.25,4,30.12


In [6]:
df_train.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y2
count,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000
mean,0.760354,674.867784,318.636872,178.115456,5.201117,3.500931,0.235940,2.854749,24.287505
std,0.104790,87.758133,43.619254,44.839207,1.750948,1.106502,0.134118,1.544532,9.505775
min,0.620000,514.500000,245.000000,110.250000,3.500000,2.000000,0.000000,0.000000,10.940000
25%,0.660000,612.500000,294.000000,147.000000,3.500000,3.000000,0.100000,2.000000,15.500000
50%,0.740000,686.000000,318.500000,220.500000,3.500000,3.000000,0.250000,3.000000,21.160000
75%,0.820000,759.500000,343.000000,220.500000,7.000000,4.000000,0.400000,4.000000,32.920000
max,0.980000,808.500000,416.500000,220.500000,7.000000,5.000000,0.400000,5.000000,48.030000


In [7]:
# Now we will extract the models input and targets from both the training and testing dataframes
def extract_Xy(df):
    df_numpy = df.to_numpy()
    return df_numpy[:, :-1], df_numpy[:, -1]

X_train, y_train = extract_Xy(df_train)
X_test, y_test = extract_Xy(df_test)

y_median = np.median(y_train)
print("Median value of the target:", y_median)

# Since we will treat this as a classification task, we will assume that
# the load is "high" (y = True) if its compressive ratio is higher than the median
# otherwise, it is assumed to be "low" (y = False)
y_train = y_train > y_median
y_test = y_test > y_median

# Now ~50% of the samples should be considered "high" and the rest are considered "low"
print(f"Percentage of 'high load' samples: {y_train.mean() * 100} %")

# Also, lets standardize the data since it improves the training process
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
X_train = (X_train - X_mean)/(1e-8 + X_std)
X_test = (X_test - X_mean)/(1e-8 + X_std)

Median value of the target: 21.16
Percentage of 'high load' samples: 49.906890130353815 %


## Logistic Regression via Scikit-Learn

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
%%time
# We use time to compute the training time of our model
model = LogisticRegression(random_state=0, penalty="none").fit(X_train, y_train)

CPU times: user 13.4 ms, sys: 0 ns, total: 13.4 ms
Wall time: 22.8 ms


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


In [10]:
from sklearn.metrics import accuracy_score

y_train_predict = model.predict(X_train)
print(f"Training Accurracy: {accuracy_score(y_train, y_train_predict) * 100}%")
y_test_predict = model.predict(X_test)
print(f"Testing Accurracy: {accuracy_score(y_test, y_test_predict) * 100}%")

Training Accurracy: 98.32402234636871%
Testing Accurracy: 96.53679653679653%


## Logistic Regression from Scratch

In [12]:
def sigmoid(x):
    #TODO: Implement sigmoid (hint: use np.exp)
    # pass
    return 1 / (1 + np.exp(-x))

In [13]:
# Sanity checks
print(f"{sigmoid(-1e2) = }") # This should be almost equal 0
print(f"{sigmoid(   0) = }") # This should be exactly 0.5
print(f"{sigmoid(+1e2) = }") # This should be almost equal 1

sigmoid(-1e2) = 3.7200759760208356e-44
sigmoid(   0) = 0.5
sigmoid(+1e2) = 1.0


In [14]:
def our_accuracy_score(true, predicted):
    #TODO: Implement an accuracy metric so that is can be used instead of Sklearn's accuracy score
    #Note: both true and predicted will be boolean numpy array
    # pass
    return np.sum(true == predicted) / len(true)

In [15]:
# Sanity checks
print(f"{our_accuracy_score( np.array([True,  True]), np.array([True,  True]) ) = }") # Should be 1
print(f"{our_accuracy_score( np.array([True, False]), np.array([True,  True]) ) = }") # Should be 0.5
print(f"{our_accuracy_score( np.array([True, False]), np.array([True, False]) ) = }") # Should be 1
print(f"{our_accuracy_score( np.array([False, True]), np.array([True, False]) ) = }") # Should be 0

our_accuracy_score( np.array([True,  True]), np.array([True,  True]) ) = 1.0
our_accuracy_score( np.array([True, False]), np.array([True,  True]) ) = 0.5
our_accuracy_score( np.array([True, False]), np.array([True, False]) ) = 1.0
our_accuracy_score( np.array([False, True]), np.array([True, False]) ) = 0.0


In [17]:
#IMPORTANT: You can only use numpy here. Do not use any premade algorithms (e.g. Scikit-Learn's Logistic Regression)
class OurLogisticRegression:
    def __init__(self, lr: int, epochs: int, probability_threshold: float = 0.5, random_state = None):
        self.lr = lr # The learning rate
        self.epochs = epochs # The number of training epochs
        self.probability_threshold = probability_threshold # If the output of the sigmoid function is > probability_threshold, the prediction is considered to be positive (True)
                                                           # otherwise, the prediction is considered to be negative (False)
        self.random_state = random_state # The random state will be used set the random seed for the sake of reproducability
    
    def _prepare_input(self, X):
        # Here, we add a new input with value 1 to each example. It will be multipled by the bias
        ones = np.ones((X.shape[0], 1), dtype=X.dtype)
        return np.concatenate((ones, X), axis=1)
    
    def _prepare_target(self, y):
        # Here, we convert True to +1 and False to -1
        #TODO (Optional): You can modify your function if you wish to used other values for the positive and negative classes
        return np.where(y, 1, -1)

    def _initialize(self, num_weights: int, stdev: float = 0.01):
        # Here, we initialize the weights using a normally distributed random variable with a small standard deviation
        self.w = np.random.randn(num_weights) * stdev

    def _gradient(self, X, y):
        #TODO: Compute and return the gradient of the weights (self.w) wrt to the loss given the X and y arrays
        # pass
        return np.dot(X.T, sigmoid(np.dot(X, self.w)) - y) / len(y)

    def _update(self, X, y):
        #TODO: Implement this function to apply a single iteration on the weights "self.w"
        #Hint: use self._gradient
        # pass
        self.w -= self.lr * self._gradient(X, y)

    def fit(self, X, y):
        np.random.seed(self.random_state) # First, we set the seed
        X = self._prepare_input(X) # Then we prepare the inputs
        y = self._prepare_target(y) # and prepare the targets too
        self._initialize(X.shape[1]) # and initialize the weights randomly
        for _ in range(self.epochs): # Then we update the weights for a certain number of epochs
            self._update(X, y)
        return self # Return self to match the behavior of Scikit-Learn's LinearRegression fit()
    
    def predict(self, X):
        X = self._prepare_input(X)
        #TODO: Implement the rest of this function (Note: It should return a boolean array)
        return sigmoid(np.dot(X, self.w)) > self.probability_threshold

In [18]:
# We will use this function to tune the hyper parameters
def validate(lr, epochs):
    validation_size = None #TODO: Choose a size for the validation set as a ratio from the training data
    X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=validation_size, random_state=42)
    # We will fit the model to only a subset of the training data and we will use the rest to evaluate the performance
    our_model = OurLogisticRegression(lr=lr, epochs=epochs, random_state=0).fit(X_tr, y_tr)
    # Then, we evaluate the peformance using the validation set
    return our_accuracy_score(y_val, our_model.predict(X_val)) 

In [24]:
lr = None #TODO: Choose a learning rate to use while testing different values for the number of epochs
epochs_values = [] #TODO: Choose a list of values for the number of epochs to test

lr = 0.5
epochs_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for epochs in epochs_values:
    accuracy = validate(lr, epochs)
    print(f"In {epochs} epochs, the accuracy reaches {accuracy * 100}% using lr={lr}")

In 1 epochs, the accuracy reaches 100.0% using lr=0.5
In 2 epochs, the accuracy reaches 100.0% using lr=0.5
In 3 epochs, the accuracy reaches 100.0% using lr=0.5
In 4 epochs, the accuracy reaches 100.0% using lr=0.5
In 5 epochs, the accuracy reaches 100.0% using lr=0.5
In 6 epochs, the accuracy reaches 100.0% using lr=0.5
In 7 epochs, the accuracy reaches 100.0% using lr=0.5
In 8 epochs, the accuracy reaches 100.0% using lr=0.5
In 9 epochs, the accuracy reaches 100.0% using lr=0.5
In 10 epochs, the accuracy reaches 100.0% using lr=0.5


In [26]:
epochs = None #TODO: Choose the number of epochs to use while testing different values for the learning rate
lr_values = [] #TODO: Choose a list of values for the learning rate to test

epochs = 10
lr_values = [0.01, 0.05, 0.1, 0.5, 1, 5, 10]
for lr in lr_values:
    accuracy = validate(lr, epochs)
    print(f"Using lr={lr}, the accuracy reaches {accuracy * 100}% in {epochs} epochs")

Using lr=0.01, the accuracy reaches 100.0% in 10 epochs
Using lr=0.05, the accuracy reaches 100.0% in 10 epochs
Using lr=0.1, the accuracy reaches 100.0% in 10 epochs
Using lr=0.5, the accuracy reaches 100.0% in 10 epochs
Using lr=1, the accuracy reaches 100.0% in 10 epochs
Using lr=5, the accuracy reaches 100.0% in 10 epochs
Using lr=10, the accuracy reaches 100.0% in 10 epochs


In [27]:
%%time
# We use time to compute the training time of our model
#TODO: Select an appropriate learning rate and number of epochs
lr = 0.1
epochs = 10
our_model = OurLogisticRegression(lr=lr, epochs=epochs, random_state=0).fit(X_train, y_train)

CPU times: user 2.01 ms, sys: 0 ns, total: 2.01 ms
Wall time: 11.6 ms


In [28]:
y_train_predict = our_model.predict(X_train)
print(f"Training Accuracy: {our_accuracy_score(y_train, y_train_predict) * 100}%")
y_test_predict = our_model.predict(X_test)
print(f"Testing Accuracy: {our_accuracy_score(y_test, y_test_predict) * 100}%")

Training Accuracy: 98.32402234636871%
Testing Accuracy: 96.53679653679653%


In [ ]:
#TODO: Write your conclusion about your implementation's performance and training time
# My implementation performance is similar to the sklearn implementation, similar accuracy 
# The training time of mine is less than the sklearn implementation due to less number of epoches